## 1. Bitcoin Market Description and Data Loading

<p>*<sup>1</sup> <strong>WARNING</strong>: The cryptocurrency market is exceptionally volatile and any money you put in might disappear into thin air.  Cryptocurrencies mentioned here <strong>might be scams</strong> similar to <a href="https://en.wikipedia.org/wiki/Ponzi_scheme">Ponzi Schemes</a> or have many other issues (overvaluation, technical, etc.). <strong>Please do not mistake this for investment advice</strong>. *</p>
<p>That said, let's get to business. As a first task, we will load the current data from the <a href="https://api.coinmarketcap.com">coinmarketcap API</a> and display it in the output.</p>

In [4]:
#Importing pandas
import pandas as pd

#Import matplotlib and setting asthetics for plotting later.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('fivethirtyeight')

#Reading in current data from coinmarketcap.com
current = pd.read_json("https://api.coinmarketcap.com/v1/ticker/")

#Printing out the first few lines
current.head()


,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,1.443987e+10,17962412,bitcoin,1569684155,146065509080,2.100000e+07,Bitcoin,-0.58,1.10,-19.01,1.000000,8131.731367,1,BTC,17962412
1,7.258250e+09,107928940,ethereum,1569684142,18481855062,NaN,Ethereum,-1.32,3.32,-20.85,0.021075,171.240957,2,ETH,107928940
2,1.021672e+09,43080011224,ripple,1569684123,10407864302,1.000000e+11,XRP,-0.33,0.94,-16.77,0.000030,0.241594,3,XRP,99991343356
3,1.755991e+10,4108044456,tether,1569684136,4159482565,NaN,Tether,0.24,0.59,0.97,0.000125,1.012521,4,USDT,4207771504
4,1.638283e+09,18027950,bitcoin-cash,1569684127,4000434581,2.100000e+07,Bitcoin Cash,-0.30,3.23,-28.95,0.027310,221.901801,5,BCH,18027950


## 2. Full dataset, filtering, and reproducibility
THe previous API call returns only the first 100 coins, and we want to explore as many coins as possible. Moreover, we can't produce reproducible analysis with live online data. To solve the problems, we will laod a CSV we conviently saved on the 6th of December of 2017 using the API call.

In [12]:
#Reading datasets/coinmarketcap_oI 6112017.csv into pandas
dec6 = pd.read_csv("datasets/coinmarketcap_06122017.csv")

#Selecting the 'id' and the 'market_cap_usd' columns
market_cap_raw = dec6[['id','market_cap_usd']]

#counting the number of values
market_cap_raw.count()

id                1326
market_cap_usd    1031
dtype: int64